# Azure OpenAI QuickStart

## Prepare the environment

### 1. Install openai if not already installed

In [5]:
%%bash python -m pip install --upgrade openai

UsageError: %%bash is a cell magic, but the cell body is empty.


### 2. Prepare project-wise environment variables

Create a `.env` text file (no filename, only `.env`) to store the api endpoints and keys. When calling `AzureOpenAI` class, the constructor will automatically load the environment variables from `.env` file.

Put `.env` in `.gitignore` to avoid committing it to git.

```text
AZURE_OPAI_ENDPOINT=https://your-endpoint.azure.com/
AZURE_OPENAI_KEY=your-key
```

## Use the Azure OpenAI API client

In [2]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = os.environ.get("AZURE_OPENAI_KEY"),  
  api_version = "2023-07-01-preview",
  azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
)

### Chat completion with a system prompt

The message parameter takes a list of dictionaries, each specifying the conversation *role* and *content*: 

- The **system** role should be included at the beginning of the list. 
- To trigger a response, end with a **user** role, which indicates that it is the **assistant**'s turn to respond.

When calling the api from Azure, the model parameter should be set to *deployment name* instead of *model name*.


In [3]:
response = client.chat.completions.create(
  # use deployment name for Azure API
  model="gpt4", 
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a three-line poem about recursion in programming."}
  ]
)

# print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

In the mirror of code, reflects a clue,
A function of self, calling anew.
In recursive tumble, endlessly it flew.


### Conversation Loop

**The model has no memory**, you need to send an updated transcript with each new question, otherwise the model will lose context of the previous questions and answers.

In [10]:
response_content = "New Conversation!"
conversation=[{"role": "system", "content": "You are a helpful assistant in academia. You are a person of few words who likes to get to the point."}]

while True:
    user_input = input(response_content + " What is your message? (q to quit)")
    print("Q: " + user_input + "\n")
    
    if user_input == "q":
        break
    
    conversation.append({"role": "user", "content": user_input})
    response = client.chat.completions.create(
        model="gpt4", # use deployment name for Azure API
        messages=conversation
    )

    response_content = response.choices[0].message.content
    conversation.append({"role": "assistant", "content": response_content})
    print("A: " + response_content + "\n" + "-" * 80 + "\n")

Q: What is curl?

A: Curl, in vector calculus, is a measure of rotation of a vector field. It's described as the "circulation density" within a region.
--------------------------------------------------------------------------------

Q: Isn't it a programming language?

A: Yes. Curl is also a command-line tool and library for transferring data using various network protocols. The term "curl" is often used to refer to this tool.
--------------------------------------------------------------------------------

Q: q

